### 频繁模式 和 关联规则 分析

In [ ]:
#-*- coding: utf-8 -*-
import csv
import numpy as np
import pandas as pd
import scipy
import math
import random
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import orangecontrib.associate.fpgrowth as oaf
from apyori import apriori
from collections import Counter
from scipy.stats import chi2_contingency

In [ ]:
# open csv file
with open('/root/dataset/wine/winemag-data-130k-v2.csv', 'r',encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    lines = list(reader)
print("This dataset has {} items.".format(len(lines)-1))
print("The index/attributes of this dataset are:")
index = lines[0]
for i in range(len(lines[0])):
    print("No.{}\t {}".format(i+1, lines[0][i]))

In [ ]:
print("Accordingly, the type of each attribute are:\n")
TYPE_NUMERIC = "numeric"
TYPE_NOMINAL = "nominal"
index_type = [1, 2, 2, 2, 1, 1, 2, 2 ,2, 2, 2, 2, 2, 2]
for item in range(len(index_type)):
    if index_type[item] == 1:
        index_type[item] = TYPE_NUMERIC
    else:
        index_type[item] = TYPE_NOMINAL
for i in range(len(index)):
    print("No.{}\t {:25}\t {}".format(i+1, index[i], index_type[i]))

In [ ]:
lines = lines[1:]

In [ ]:
ItemNo = [x+1 for x in range(len(lines))]
df = pd.DataFrame(lines, index=ItemNo, columns=index)
df.describe()

In [ ]:
print("Initializing lists...")
id = []
country = []
description = []
designation =[]
points = []
price = []
province = []
region_1 = []
region_2 = []
taster_name = []
taster_twitter_handle = []
title = []
variety = []
winery = []

In [ ]:
for item in lines:
    id.append(item[0])
    country.append(item[1])
    description.append(item[2])
    designation.append(item[3])
    points.append(item[4])
    price.append(item[5])
    province.append(item[6])
    region_1.append(item[7])
    region_2.append(item[8])
    taster_name.append(item[9])
    taster_twitter_handle.append(item[10])
    title.append(item[11])
    variety.append(item[12])
    winery.append(item[13])
print("Initialize lists finished.")

About this dataset, we analysis the frequent mode among country/points/price/province/region_1/region_2/winery attributes.

In [ ]:
Data_set = []
for i in range(len(country)):
    if country[i] != '' and points[i] != '' and price[i] != '' and province[i] != '' and winery[i] != '':
        item = []
        item.append(country[i])
        item.append(points[i])
        item.append(price[i])
        item.append(province[i])
        #item.append(region_1[i])
        #item.append(region_2[i])
        item.append(winery[i])
        Data_set.append(item)
print("Initialized data set for analysis.")
print(len(Data_set))

In [ ]:
result = list(apriori(transactions=Data_set))
for item in result:
    print(item)
    print()
print(type(result[0]))

出现的频繁项集有：

In [ ]:
count = 0
item_len = 1
len_list = []
for item in result:
    if len(item[0]) == item_len:
        count += 1
    else:
        print(str(item_len)+"项集有"+str(count)+"个。\n")
        len_list.append(count)
        count = 1
        item_len += 1
    print(item[0])
print(str(item_len)+"项集有"+str(count)+"个。\n")
len_list.append(count)

单一项集分析：

In [ ]:
Name_list = []
Support_list = []
Confidence_list = []
Lift_list = []
K2_list = []
len_items = len_list[0]
for item in range(len_items):
    for name in result[item][0]:
        Name_list.append(name)
    Support_list.append(result[item][1])
    Confidence_list.append(result[item][2][0][2])
    Lift_list.append(result[item][2][0][3])

In [ ]:
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['figure.dpi'] = 150
plt.rcParams['figure.figsize'] = (10.0, 4.0)

x_range = list(range(len(Name_list)))

rects1 = plt.bar(x=x_range, height=Support_list, width=0.25, color='red', label="Support")
rects2 = plt.bar(x=[i + 0.25 for i in x_range], height=Confidence_list, width=0.25, color='lightgreen', label="Confidence")
rects3 = plt.bar(x=[i + 0.5 for i in x_range], height=Lift_list, width=0.25, color='lightblue', label="Lift")
plt.ylim(0, 1)     # y轴取值范围

plt.xticks([index + 0.2 for index in x_range], Name_list)
plt.xlabel("Mode Name")
plt.legend()     # 设置题注
plt.show()

In [ ]:
for item in range(len(Name_list)):
    print("【"+str(Name_list[item])+"】")
    print(str(Name_list[item])+"-"+"Support:"+str(Support_list[item]))
    print(str(Name_list[item])+"-"+"Confidence:"+str(Confidence_list[item]))
    print(str(Name_list[item])+"-"+"Lift:"+str(Lift_list[item]))
    print()

二项集分析：

In [ ]:
item=result[7]
Name_list = ["US - California", "California <- US", "US <- California"]
Support_list = [item[1], 0, 0]
Confidence_list = [item[2][0][2], item[2][1][2], item[2][2][2]]
Lift_list = [item[2][0][3], item[2][1][3], item[2][2][3]]

In [ ]:
plt.clf()
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['figure.dpi'] = 150
plt.rcParams['figure.figsize'] = (10.0, 4.0)

x_range = list(range(len(Name_list)))

plt.bar(x=x_range, height=Support_list, width=0.25, color='red', label="Support")
plt.bar(x=[i + 0.25 for i in x_range], height=Confidence_list, width=0.25, color='lightgreen', label="Confidence")
plt.bar(x=[i + 0.5 for i in x_range], height=Lift_list, width=0.25, color='lightblue', label="Lift")

plt.xticks([index + 0.2 for index in x_range], Name_list)
plt.xlabel("Mode Name")
plt.legend()     # 设置题注
plt.show()

In [ ]:
for item in range(len(Name_list)):
    print("【"+str(Name_list[item])+"】")
    print(str(Name_list[item])+"-"+"Support:"+str(Support_list[item]))
    print(str(Name_list[item])+"-"+"Confidence:"+str(Confidence_list[item]))
    print(str(Name_list[item])+"-"+"Lift:"+str(Lift_list[item]))
    print()

卡方分析检验：  
分析产地对就价格的影响：

In [ ]:
Data_list1 = []
Data_list2 = []
price = np.array(price)
# print("converted")
for item in range(len(province)):
    if province[item] == "California" and price[item] != '':
        Data_list1.append(float(price[item]))
    elif province[item] == "Washington" and price[item] != '':
        Data_list2.append(float(price[item]))
len_items = min(len(Data_list1), len(Data_list2))
if len(Data_list1) == len_items:
    Data_list2 = random.sample(Data_list2, len_items)
else:
    Data_list1 = random.sample(Data_list1, len_items)

Data_list1 = np.array(Data_list1).astype(int)
Data_list2 = np.array(Data_list2).astype(int)
obs = np.array([Data_list1, Data_list2])
chi2_result = chi2_contingency(obs)

产地 California 与 Washington 对价格影响的卡方为：

In [ ]:
print(chi2_result[0])

其它类似卡方检验可以使用以上同样方式进行。